https://python.langchain.com/docs/use_cases/sql/quickstart/

### **Test the sqldb**

In [2]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

**Connecting to the sqldb**

In [3]:
db_path = str(here("data")) + "/sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [3]:
db

In [ ]:
# validate the connection to the database
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Customer LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3), (2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5), (3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3), (4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4), (5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4), (6, 'Helena', 'Holý', None, 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', '+420 2 4177 0449', None, 'hholy@gmail.com', 5), (7, 'As

### **Test your GPT model**

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [2]:
def get_response(prompt):

    from groq import Groq    
    groq_client = Groq(api_key=GROQ_API_KEY)
    
    completion = groq_client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {
                "role": "system",
                "content": "Assistant",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        temperature=0.1,
        max_tokens=8000,
        top_p=1,
        stream=False,
        stop=None,
    )    
    response = completion.choices[0].message.content    
    return response

In [ ]:
prompt = "Tell me a story about Elene in 5 sentences"
get_response(prompt)

### **1. SQL query chain**

In [ ]:
# Load the LLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(model_name="llama-3.1-70b-versatile",temperature=0.0)

In [28]:
from langchain.chains import create_sql_query_chain
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many customers are there"})
print(response)

Question: How many customers are there
SQLQuery: SELECT COUNT("CustomerId") FROM "Customer"


In [52]:
sql_query = response.split("SQLQuery: ")[1].strip()
db.run(sql_query)


'[(59,)]'

In [38]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

Execute the query to make sure it’s valid

### **Add QuerySQLDataBaseTool to the chain**
Execute SQL query

**This is the most dangerous part of creating a SQL chain.** Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [ ]:
### not working !!!!
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

result = chain.invoke({"question": "How many employees are there"})
print(result)

In [67]:
### from Claude

from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

# Get the SQL query first
query_response = write_query.invoke({"question": "How many employees are there"})

# Extract just the SQL query part
sql_query = query_response.split("SQLQuery: ")[1].strip()
print("SQL Query:", sql_query)

# Execute the query directly with the tool
result = execute_query.run(sql_query)
print("Result:", result)

SQL Query: SELECT COUNT("EmployeeId") FROM "Employee"
Result: [(8,)]


### **Answer the question in a user friendly manner**

In [ ]:
### not working !!!!
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQLQuery: {query}
SQLResult: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

In [96]:
### from Claude
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# First, create a function to clean the SQL query
def clean_sql_query(response):
    return response.split("SQLQuery: ")[1].strip()

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQLQuery: {query}
SQLResult: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()

write_query = create_sql_query_chain(llm, db)

# Modify the chain to clean the SQL query before execution
chain = (
    RunnablePassthrough.assign(
        query=write_query
    ).assign(
        result=lambda x: execute_query.run(clean_sql_query(x["query"]))
    )
    | answer
)

result = chain.invoke({"question": """how many customers whose first name starts with M bought albums from genre Pop, 
                       list them, and what album they bought
                        """})
# result = chain.invoke({"question": """list all customers whose first name starts with K bought albums from genre Pop, 
#                        provide: first name, last name and country """})
print(result)

Based on the SQL query and result, there is only 1 customer whose first name starts with 'M' who bought albums from the genre Pop. 

The customer's name is Marc Dubois, and he bought the album 'Axé Bahia 2001' (twice, as indicated by the duplicate entry in the result).


In [ ]:
chain

### **2. Agents**

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

- It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can answer questions that require multiple dependent queries.
- It will save tokens by only considering the schema from relevant tables.

To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions
- …

In [108]:
from langchain_community.agent_toolkits import create_sql_agent

# agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
agent_executor = create_sql_agent(llm, db=db, agent_type="tool-calling", verbose=True)


In [109]:
agent_executor.invoke(
    {
        "input": "List the total sales per country, sorted from highest to lowest."
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`
responded:   Then I should double check my query before executing it.



Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'orders, customers, employees, products, order_details'}`
responded:   Then I should double check my query before executing it.



Error: table_names {'products', 'order_details', 'employees', 'customers', 'orders'} not found in database
Invoking: `sql_db_query_checker` with `{'query': 'SELECT T1.Country, SUM(T2.UnitPrice * T2.Quantity) AS TotalSales FROM customers AS T1 INNER JOIN orders AS T3 ON T1.CustomerID = T3.CustomerID INNER JOIN `order details` AS T2 ON T3.OrderID = T2.OrderID GROUP BY T1.Country ORDER BY TotalSales DESC LIMIT 10;'}`
responded:   Then I should double check my query before executing it.



SELECT T1.Country, SUM(T2.UnitPrice * T2.Quantity) AS TotalSales

{'input': 'List the total sales per country, sorted from highest to lowest.',
 'output': 'The total sales per country, sorted from highest to lowest, are:\n\n1. USA - $523.06\n2. Canada - $303.96\n3. France - $195.10\n4. Brazil - $190.10\n5. Germany - $156.48\n6. United Kingdom - $112.86\n7. Czech Republic - $90.24\n8. Portugal - $77.24\n9. India - $75.26\n10. Chile - $46.62'}

In [ ]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})
# agent_executor.invoke("Describe the playlisttrack table")